# Android Security Bulletin Data Download

This notebook downloads data on CVEs reported on the monthly Android Security Bulletin, and merges it with the data provided by the CVE Program and with the data found on the AOSP git repo. The resulting aggregated data contains information about the dates of events of interest (e.g. commit to the repo, publication on the bulletin) for every patch published on the bulletin.

In [1]:
import requests
import re
import dateutil
import json
from bs4 import BeautifulSoup
import xml.etree.ElementTree as ET
import pandas as pd
from os import path, listdir


BASE_URL = "https://source.android.com"
LIST_URL = "https://source.android.com/docs/security/bulletin/asb-overview"
CVE_REGEX = re.compile("CVE-\d+-\d+")

In [2]:
# page = requests.get(LIST_URL)
#
# doc = BeautifulSoup(page.content, "html.parser")
#
# rows = doc.find_all("tr")
#
# urls = [ BASE_URL + row.find("td").find("a")["href"] for row in rows[1:]]
#
# # filter urls
# urls = [ u for u in urls if path.basename(u) < "2023" and path.basename(u) >= "2017-06-01"]

In [3]:
from dateutil import parser
def get_url_and_date(url_page):
  ret = []
  page = requests.get(url_page)
  doc = BeautifulSoup(page.content, "html.parser")
  rows = doc.find_all("tr")
  for row in rows[1:]:
    url = BASE_URL + row.find("td").find("a")["href"]
    date = parser.parse(row.find_all("td")[2].get_text().strip())
    ret.append((url, date))
  return ret

urls = get_url_and_date(LIST_URL)

# filter urls
urls = [ (u, d) for u, d in urls if path.basename(u) < "2025" and path.basename(u) >= "2017-06-01"]
urls

[('https://source.android.com/docs/security/bulletin/2024-12-01',
  datetime.datetime(2024, 12, 2, 0, 0)),
 ('https://source.android.com/docs/security/bulletin/2024-11-01',
  datetime.datetime(2024, 11, 4, 0, 0)),
 ('https://source.android.com/docs/security/bulletin/2024-10-01',
  datetime.datetime(2024, 10, 7, 0, 0)),
 ('https://source.android.com/docs/security/bulletin/2024-09-01',
  datetime.datetime(2024, 9, 3, 0, 0)),
 ('https://source.android.com/docs/security/bulletin/2024-08-01',
  datetime.datetime(2024, 8, 5, 0, 0)),
 ('https://source.android.com/docs/security/bulletin/2024-07-01',
  datetime.datetime(2024, 7, 1, 0, 0)),
 ('https://source.android.com/docs/security/bulletin/2024-06-01',
  datetime.datetime(2024, 6, 3, 0, 0)),
 ('https://source.android.com/docs/security/bulletin/2024-05-01',
  datetime.datetime(2024, 5, 6, 0, 0)),
 ('https://source.android.com/docs/security/bulletin/2024-04-01',
  datetime.datetime(2024, 4, 1, 0, 0)),
 ('https://source.android.com/docs/security

In [4]:
! wget https://cve.mitre.org/data/downloads/allitems-cvrf.xml

--2025-01-19 23:43:39--  https://cve.mitre.org/data/downloads/allitems-cvrf.xml
Resolving cve.mitre.org (cve.mitre.org)... 192.52.194.205, 198.49.146.205
Connecting to cve.mitre.org (cve.mitre.org)|192.52.194.205|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 413168401 (394M) [text/xml]
Saving to: ‘allitems-cvrf.xml’

allitems-cvrf.xml   100%[===================>] 394.03M  43.8MB/s    in 13s     

2025-01-19 23:43:53 (30.6 MB/s) - ‘allitems-cvrf.xml’ saved [413168401/413168401]



In [5]:
! wget https://nvd.nist.gov/feeds/json/cve/1.1/nvdcve-1.1-2012.json.gz
! wget https://nvd.nist.gov/feeds/json/cve/1.1/nvdcve-1.1-2013.json.gz
! wget https://nvd.nist.gov/feeds/json/cve/1.1/nvdcve-1.1-2014.json.gz
! wget https://nvd.nist.gov/feeds/json/cve/1.1/nvdcve-1.1-2015.json.gz
! wget https://nvd.nist.gov/feeds/json/cve/1.1/nvdcve-1.1-2016.json.gz
! wget https://nvd.nist.gov/feeds/json/cve/1.1/nvdcve-1.1-2017.json.gz
! wget https://nvd.nist.gov/feeds/json/cve/1.1/nvdcve-1.1-2018.json.gz
! wget https://nvd.nist.gov/feeds/json/cve/1.1/nvdcve-1.1-2019.json.gz
! wget https://nvd.nist.gov/feeds/json/cve/1.1/nvdcve-1.1-2020.json.gz
! wget https://nvd.nist.gov/feeds/json/cve/1.1/nvdcve-1.1-2021.json.gz
! wget https://nvd.nist.gov/feeds/json/cve/1.1/nvdcve-1.1-2022.json.gz
! wget https://nvd.nist.gov/feeds/json/cve/1.1/nvdcve-1.1-2023.json.gz
! wget https://nvd.nist.gov/feeds/json/cve/1.1/nvdcve-1.1-2024.json.gz
! gzip -dkf nvdcve-1.1-2024.json.gz
! gzip -dkf nvdcve-1.1-2023.json.gz
! gzip -dkf nvdcve-1.1-2022.json.gz
! gzip -dkf nvdcve-1.1-2021.json.gz
! gzip -dkf nvdcve-1.1-2020.json.gz
! gzip -dkf nvdcve-1.1-2019.json.gz
! gzip -dkf nvdcve-1.1-2018.json.gz
! gzip -dkf nvdcve-1.1-2017.json.gz
! gzip -dkf nvdcve-1.1-2016.json.gz
! gzip -dkf nvdcve-1.1-2015.json.gz
! gzip -dkf nvdcve-1.1-2014.json.gz
! gzip -dkf nvdcve-1.1-2013.json.gz
! gzip -dkf nvdcve-1.1-2012.json.gz

--2025-01-19 23:43:53--  https://nvd.nist.gov/feeds/json/cve/1.1/nvdcve-1.1-2012.json.gz
Resolving nvd.nist.gov (nvd.nist.gov)... 18.235.227.114, 2600:1f18:268d:1d01:f609:5e91:8a48:f546
Connecting to nvd.nist.gov (nvd.nist.gov)|18.235.227.114|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 2122993 (2.0M) [application/x-gzip]
Saving to: ‘nvdcve-1.1-2012.json.gz’

nvdcve-1.1-2012.jso 100%[===================>]   2.02M  5.35MB/s    in 0.4s    

2025-01-19 23:43:54 (5.35 MB/s) - ‘nvdcve-1.1-2012.json.gz’ saved [2122993/2122993]

--2025-01-19 23:43:54--  https://nvd.nist.gov/feeds/json/cve/1.1/nvdcve-1.1-2013.json.gz
Resolving nvd.nist.gov (nvd.nist.gov)... 18.235.227.114, 2600:1f18:268d:1d01:f609:5e91:8a48:f546
Connecting to nvd.nist.gov (nvd.nist.gov)|18.235.227.114|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 2488847 (2.4M) [application/x-gzip]
Saving to: ‘nvdcve-1.1-2013.json.gz’

nvdcve-1.1-2013.jso 100%[===================>]   2.3

In [6]:
! wget https://github.com/CVEProject/cvelistV5/archive/refs/heads/main.zip
! unzip main.zip

Streaming output truncated to the last 5000 lines.
  inflating: cvelistV5-main/cves/2024/5xxx/CVE-2024-5383.json  
  inflating: cvelistV5-main/cves/2024/5xxx/CVE-2024-5384.json  
  inflating: cvelistV5-main/cves/2024/5xxx/CVE-2024-5385.json  
  inflating: cvelistV5-main/cves/2024/5xxx/CVE-2024-5387.json  
  inflating: cvelistV5-main/cves/2024/5xxx/CVE-2024-5388.json  
  inflating: cvelistV5-main/cves/2024/5xxx/CVE-2024-5389.json  
  inflating: cvelistV5-main/cves/2024/5xxx/CVE-2024-5390.json  
  inflating: cvelistV5-main/cves/2024/5xxx/CVE-2024-5391.json  
  inflating: cvelistV5-main/cves/2024/5xxx/CVE-2024-5392.json  
  inflating: cvelistV5-main/cves/2024/5xxx/CVE-2024-5393.json  
  inflating: cvelistV5-main/cves/2024/5xxx/CVE-2024-5394.json  
  inflating: cvelistV5-main/cves/2024/5xxx/CVE-2024-5395.json  
  inflating: cvelistV5-main/cves/2024/5xxx/CVE-2024-5396.json  
  inflating: cvelistV5-main/cves/2024/5xxx/CVE-2024-5397.json  
  inflating: cvelistV5-main/cves/2024/5xxx/CVE-2024-5

In [7]:
ns = {
    "vuln": "http://www.icasi.org/CVRF/schema/vuln/1.1",
    "cvrf": "http://www.icasi.org/CVRF/schema/cvrf/1.1",
}

def load_cvrf(filename):
  tree = ET.parse(filename)
  doc = tree.getroot()
  vulns = doc.findall("vuln:Vulnerability", namespaces=ns)
  ret = []
  for vuln in vulns:
    name = vuln.find("vuln:Title", namespaces=ns).text
    published = vuln.find("./vuln:Notes/vuln:Note[@Title='Published']", namespaces=ns)
    if published is not None:
      published = published.text
    modified = vuln.find("./vuln:Notes/vuln:Note[@Title='Modified']", namespaces=ns)
    if modified is not None:
      modified = modified.text
    ret.append({"cve": name, "published": published, "modified": modified})
  return pd.DataFrame(ret)

def load_cvrf_json(years):
  ret = []
  for y in years:
    dirs = listdir(f"cvelistV5-main/cves/{y}")
    for d in dirs:
      jsons = listdir(f"cvelistV5-main/cves/{y}/{d}")
      for j in jsons:
        with open(f"cvelistV5-main/cves/{y}/{d}/{j}", "r") as f:
          payload = json.load(f)
        try:
          published = payload["cveMetadata"]["datePublished"] if "datePublished" in payload["cveMetadata"] else None
          ret.append({"cve": payload["cveMetadata"]["cveId"], "published": published , "modified": payload["cveMetadata"]["dateUpdated"]})
        except Exception as e:
          print(e)
          print(payload)
          pass
  return pd.DataFrame(ret)


def get_modified(df, id):
  return df[df["cve"] == id].iloc[0].at["modified"]

def get_published(df, id):
  return df[df["cve"] == id].iloc[0].at["published"]

In [8]:
# TODO: use this to filter Acknoledgements table?
def filter_table(table):
  cur = table.previous_sibling
  while cur.name != "h2":
    cur = cur.previous_sibling
  return cur.get_text().strip().lower() == "acknowledgements"

In [9]:
def print_rows(rows):
  for row in rows:
    print(row.get_text())

def filter_rows(rows):
  return [row for row in rows if CVE_REGEX.search(row.get_text()) is not None]

def get_cves(info, get_commits=True):
  url, bulletin_date = info
  cves = []
  patch_level = path.basename(url)
  page = requests.get(url)
  doc = BeautifulSoup(page.content, "html.parser")
  tables = doc.find_all("table")
  #tables = [t for t in tables if not filter_table(t)]
  for table in tables:
    category = table.find_previous("h3").get_text().strip().lower()
    rows = filter_rows(table.find_all("td"))
    rows2 = table.find_all("td", string=CVE_REGEX)
    for row in rows:
      cells = row.parent.find_all("td")
      if category.startswith("google play"):
        cve = cells[1].get_text().strip()
        print(cve)
      else:
        cve = cells[0].get_text().strip()
      refs = []
      if get_commits:
        links = cells[1].find_all("a")
        if links is not None and CVE_REGEX.search(cve) is not None:
          for a in links:
            try:
              link = a["href"]
              (created, committed) = get_commit_date(link)
              ref = {"link": link, "created": created, "committed": committed}
              refs.append(ref)
            except:
              pass
      cur_cves = cve.split(',')
      for c in cur_cves:
        cves.append({"cve": c.strip(), "refs": refs, "bulletin_date": bulletin_date, "category": category, "patch_level": patch_level })
  return cves

def get_date_from_row(doc, head):
  try:
    row = doc.find("th", string=head).parent.find_all("td")
    return dateutil.parser.parse(row[1].text)
  except:
    return None

# matches codeaurora urls, making a distiction between cgit urls and gitweb urls.
# in case an url is a cgit url (i.e. group 1 matches), group 2 contains the path
# of the repo and group 3 contains the commit id.
# In case an url is a gitweb url, group 4 contains the path (including a '?p=...'
# parameter that shouuld be removed) and group 5 contains the commit id
CODEAURORA_RE = re.compile(r"https:\/\/(source\.codeaurora\.org|us\.codeaurora\.org\/cgit)\/quic\/(.*)\/commit\/?\?.*id=(.*)|https:\/\/www\.codeaurora\.org\/gitweb\/quic\/(.*\/?p=.*)\.git;a=commit;h=(.*)")

def fix_codeaurora_url(url):
  matches = CODEAURORA_RE.match(url)
  if matches:
    if matches.group(1):
      path = matches.group(2)
      commit = matches.group(3)
    else:
      path = matches.group(4).replace('?p=', '')
      commit = matches.group(5)
    url = f"https://git.codelinaro.org/clo/{path}/-/commit/{commit}"
  return url

# github web pages do not have all the information we can extract from cgit, so
# we can use the url for the old repo
def fix_github_url(url):
  return url.replace("github.com/torvalds/linux/commit/", "git.kernel.org/cgit/linux/kernel/git/torvalds/linux.git/commit/?id=")

def fix_url(url):
  url = fix_codeaurora_url(url)
  url = fix_github_url(url)
  return url

def get_codelinaro_date(doc):
  try:
    times = doc.findAll("time")
    return (dateutil.parser.parse(times[0]["datetime"]), dateutil.parser.parse(times[1]["datetime"]))
  except:
    return (None, None)

def get_commit_date(url):
  url = fix_url(url)
  print(f"\tget commit date from {url}...")
  page = requests.get(url)
  doc = BeautifulSoup(page.content, "html.parser")
  if "git.codelinaro.org" in url:
    return get_codelinaro_date(doc)
  return (get_date_from_row(doc, "author"), get_date_from_row(doc, "committer"))


In [10]:
cves = []
for url in set(urls):
  print(f"download {url}...")
  cves.extend(get_cves(url))

Streaming output truncated to the last 5000 lines.
	get commit date from #asterisk...
	get commit date from #asterisk...
	get commit date from #asterisk...
	get commit date from #asterisk...
	get commit date from #asterisk...
	get commit date from #asterisk...
	get commit date from #asterisk...
	get commit date from #asterisk...
CVE-2021-0971
download ('https://source.android.com/docs/security/bulletin/2024-04-01', datetime.datetime(2024, 4, 1, 0, 0))...
	get commit date from https://android.googlesource.com/platform/frameworks/base/+/c7c1193ca7554fc3182ca9ab0f711bd118bdeef2...
	get commit date from https://android.googlesource.com/platform/frameworks/base/+/ffd616dc3b919fe7705dbc7a25868483ae45466b...
	get commit date from https://android.googlesource.com/platform/frameworks/base/+/bdf1cce569c9700965ff6baee8efd3fb1e8269e8...
	get commit date from https://android.googlesource.com/platform/frameworks/base/+/6beb68ca17d1220f3f09a53cf0a0c541db4ead62...
	get commit date from https://android

In [11]:
urls

[('https://source.android.com/docs/security/bulletin/2024-12-01',
  datetime.datetime(2024, 12, 2, 0, 0)),
 ('https://source.android.com/docs/security/bulletin/2024-11-01',
  datetime.datetime(2024, 11, 4, 0, 0)),
 ('https://source.android.com/docs/security/bulletin/2024-10-01',
  datetime.datetime(2024, 10, 7, 0, 0)),
 ('https://source.android.com/docs/security/bulletin/2024-09-01',
  datetime.datetime(2024, 9, 3, 0, 0)),
 ('https://source.android.com/docs/security/bulletin/2024-08-01',
  datetime.datetime(2024, 8, 5, 0, 0)),
 ('https://source.android.com/docs/security/bulletin/2024-07-01',
  datetime.datetime(2024, 7, 1, 0, 0)),
 ('https://source.android.com/docs/security/bulletin/2024-06-01',
  datetime.datetime(2024, 6, 3, 0, 0)),
 ('https://source.android.com/docs/security/bulletin/2024-05-01',
  datetime.datetime(2024, 5, 6, 0, 0)),
 ('https://source.android.com/docs/security/bulletin/2024-04-01',
  datetime.datetime(2024, 4, 1, 0, 0)),
 ('https://source.android.com/docs/security

In [12]:
import numpy as np

df = pd.DataFrame(data=cves)
df_cat = df[df["bulletin_date"] >= "2017-06-01"]
df_cat.groupby("category")["bulletin_date"].count()

,bulletin_date
category,
amlogic,3
amlogic components,2
android runtime,20
android tv,2
arm components,46
bluetooth,2
broadcom components,22
fpc components,9
framework,478


In [13]:
def load_nvd(filename):
  data = dict()
  with open(filename, "r") as f:
    payload = json.load(f)
    for item in payload["CVE_Items"]:
      id = item["cve"]["CVE_data_meta"]["ID"]
      data[id] = item
  return data

In [14]:
nvd_files = [ f for f in listdir() if path.isfile(f) and f.startswith("nvdcve") and f.endswith(".json")]

nvds = dict()
cve_meta = load_cvrf_json(range(2016, 2025))

for filename in nvd_files:
  print(f"loading {filename}")
  nvds = {**nvds, **load_nvd(filename)}

loading nvdcve-1.1-2023.json
loading nvdcve-1.1-2012.json
loading nvdcve-1.1-2018.json
loading nvdcve-1.1-2024.json
loading nvdcve-1.1-2013.json
loading nvdcve-1.1-2022.json
loading nvdcve-1.1-2017.json
loading nvdcve-1.1-2015.json
loading nvdcve-1.1-2021.json
loading nvdcve-1.1-2014.json
loading nvdcve-1.1-2019.json
loading nvdcve-1.1-2020.json
loading nvdcve-1.1-2016.json


In [15]:
import traceback

to_remove = []

for cve in cves:
  id = cve["cve"]
  try:
    mod = get_modified(cve_meta, id)
    pub = get_published(cve_meta, id)
    cve["cve_modified"] = mod
    cve["cve_published"] = pub
    if id in nvds and nvds[id]["impact"]:
      if "baseMetricV2" in nvds[id]["impact"]:
        impact = nvds[id]["impact"]["baseMetricV2"]
        score = impact["cvssV2"]["baseScore"]
        severity = impact["severity"]
        cve["nvd_severity"] = severity
        cve["nvd_score"] = score
      if "baseMetricV3" in nvds[id]["impact"]:
        impact = nvds[id]["impact"]["baseMetricV3"]
        cve["nvd_score_v3"] = impact["cvssV3"]["baseScore"]
        cve["nvd_severity_v3"] = impact["cvssV3"]["baseSeverity"]
    else:
      cve["nvd_severity"] = None
      cve["nvd_score"] = None
      print(f"skipping NVD for {id}...")
  except Exception as e:
    traceback.print_exc()
    print(f"error with cve {id}, skipping")
    # to_remove.append(cve)

for cve in to_remove:
  cves.remove(cve)

Traceback (most recent call last):
  File "<ipython-input-15-5dfe7850ec07>", line 8, in <cell line: 0>
    mod = get_modified(cve_meta, id)
          ^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "<ipython-input-7-3d00f02476e8>", line 42, in get_modified
    return df[df["cve"] == id].iloc[0].at["modified"]
           ~~~~~~~~~~~~~~~~~~~~~~~~^^^
  File "/usr/local/lib/python3.11/dist-packages/pandas/core/indexing.py", line 1191, in __getitem__
    return self._getitem_axis(maybe_callable, axis=axis)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.11/dist-packages/pandas/core/indexing.py", line 1752, in _getitem_axis
    self._validate_integer(key, axis)
  File "/usr/local/lib/python3.11/dist-packages/pandas/core/indexing.py", line 1685, in _validate_integer
    raise IndexError("single positional indexer is out-of-bounds")
IndexError: single positional indexer is out-of-bounds


error with cve CVE-2021-1978, skipping
skipping NVD for CVE-2018-11816...
skipping NVD for CVE-2024-20039...
skipping NVD for CVE-2024-20040...
skipping NVD for CVE-2023-28582...
skipping NVD for CVE-2023-33084...
skipping NVD for CVE-2023-33086...
skipping NVD for CVE-2023-33095...
skipping NVD for CVE-2023-33096...
skipping NVD for CVE-2023-33099...
skipping NVD for CVE-2023-33100...
skipping NVD for CVE-2023-33101...
skipping NVD for CVE-2023-33103...
skipping NVD for CVE-2023-33104...
skipping NVD for CVE-2023-33115...
skipping NVD for CVE-2022-2209...
skipping NVD for CVE-2022-2209...
skipping NVD for CVE-2022-20463...
skipping NVD for CVE-2023-6143...
skipping NVD for CVE-2023-6241...
skipping NVD for CVE-2024-20005...
skipping NVD for CVE-2024-20022...
skipping NVD for CVE-2024-20023...
skipping NVD for CVE-2024-20024...
skipping NVD for CVE-2024-20025...
skipping NVD for CVE-2024-20027...
skipping NVD for CVE-2024-20028...
skipping NVD for CVE-2024-20020...
skipping NVD for CVE

Traceback (most recent call last):
  File "<ipython-input-15-5dfe7850ec07>", line 8, in <cell line: 0>
    mod = get_modified(cve_meta, id)
          ^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "<ipython-input-7-3d00f02476e8>", line 42, in get_modified
    return df[df["cve"] == id].iloc[0].at["modified"]
           ~~~~~~~~~~~~~~~~~~~~~~~~^^^
  File "/usr/local/lib/python3.11/dist-packages/pandas/core/indexing.py", line 1191, in __getitem__
    return self._getitem_axis(maybe_callable, axis=axis)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.11/dist-packages/pandas/core/indexing.py", line 1752, in _getitem_axis
    self._validate_integer(key, axis)
  File "/usr/local/lib/python3.11/dist-packages/pandas/core/indexing.py", line 1685, in _validate_integer
    raise IndexError("single positional indexer is out-of-bounds")
IndexError: single positional indexer is out-of-bounds


error with cve CVE-2022-25704, skipping


Traceback (most recent call last):
  File "<ipython-input-15-5dfe7850ec07>", line 8, in <cell line: 0>
    mod = get_modified(cve_meta, id)
          ^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "<ipython-input-7-3d00f02476e8>", line 42, in get_modified
    return df[df["cve"] == id].iloc[0].at["modified"]
           ~~~~~~~~~~~~~~~~~~~~~~~~^^^
  File "/usr/local/lib/python3.11/dist-packages/pandas/core/indexing.py", line 1191, in __getitem__
    return self._getitem_axis(maybe_callable, axis=axis)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.11/dist-packages/pandas/core/indexing.py", line 1752, in _getitem_axis
    self._validate_integer(key, axis)
  File "/usr/local/lib/python3.11/dist-packages/pandas/core/indexing.py", line 1685, in _validate_integer
    raise IndexError("single positional indexer is out-of-bounds")
IndexError: single positional indexer is out-of-bounds
Traceback (most recent call last):
  File "<ipython-input-15-5dfe7850ec07>", line 

...

error with cve CVE-2013-4397, skipping
error with cve 1.1, skipping
error with cve 1.3, skipping
skipping NVD for CVE-2018-9409...
error with cve CVE-2018-9373, skipping


Traceback (most recent call last):
  File "<ipython-input-15-5dfe7850ec07>", line 8, in <cell line: 0>
    mod = get_modified(cve_meta, id)
          ^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "<ipython-input-7-3d00f02476e8>", line 42, in get_modified
    return df[df["cve"] == id].iloc[0].at["modified"]
           ~~~~~~~~~~~~~~~~~~~~~~~~^^^
  File "/usr/local/lib/python3.11/dist-packages/pandas/core/indexing.py", line 1191, in __getitem__
    return self._getitem_axis(maybe_callable, axis=axis)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.11/dist-packages/pandas/core/indexing.py", line 1752, in _getitem_axis
    self._validate_integer(key, axis)
  File "/usr/local/lib/python3.11/dist-packages/pandas/core/indexing.py", line 1685, in _validate_integer
    raise IndexError("single positional indexer is out-of-bounds")
IndexError: single positional indexer is out-of-bounds


skipping NVD for CVE-2018-9372...


Traceback (most recent call last):
  File "<ipython-input-15-5dfe7850ec07>", line 8, in <cell line: 0>
    mod = get_modified(cve_meta, id)
          ^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "<ipython-input-7-3d00f02476e8>", line 42, in get_modified
    return df[df["cve"] == id].iloc[0].at["modified"]
           ~~~~~~~~~~~~~~~~~~~~~~~~^^^
  File "/usr/local/lib/python3.11/dist-packages/pandas/core/indexing.py", line 1191, in __getitem__
    return self._getitem_axis(maybe_callable, axis=axis)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.11/dist-packages/pandas/core/indexing.py", line 1752, in _getitem_axis
    self._validate_integer(key, axis)
  File "/usr/local/lib/python3.11/dist-packages/pandas/core/indexing.py", line 1685, in _validate_integer
    raise IndexError("single positional indexer is out-of-bounds")
IndexError: single positional indexer is out-of-bounds


error with cve 1.2, skipping
skipping NVD for CVE-2024-36972...
skipping NVD for CVE-2024-3655...
skipping NVD for CVE-2024-33035...
skipping NVD for CVE-2024-23358...
skipping NVD for CVE-2024-23359...
skipping NVD for CVE-2024-23362...
skipping NVD for CVE-2024-23364...
skipping NVD for CVE-2024-23365...
skipping NVD for CVE-2024-33016...


In [16]:
output = "drive/My Drive/cves_temp/cves-severity-google-play.json"

with open(output, "w") as f:
  json.dump(cves, f, default=str)

In [17]:
import csv

def to_csv(cves, out):
  """
  Converts the raw cves to a csv file. The various commit dates are aggregated into
  `commit_start` and `commit_end`, which are respectively the earliest and the
  latest dates that appear in the various commits.
  Duplicated CVEs are left as is, and should be handled in another preprocessing
  step
  """
  with open(out, "w") as csvfile:
    w = csv.writer(csvfile)
    w.writerow(["id", "bulletin_date", "patch_level", "category", "published", "modified", "commit_start", "commit_end", "nvd_severity", "nvd_score", "nvd_severity_v3", "nvd_score_v3",])
    for cve in cves:
      dates = [d for r in cve["refs"] for d in [r["committed"], r["created"]] if r["committed"] is not None and r["created"] is not None]
      if dates:
        start = min(dates)
        end = max(dates)
      else:
        start = None
        end = None
      try:
        w.writerow([cve["cve"], cve["bulletin_date"], cve["patch_level"], cve.get("category"), cve.get("cve_published"), cve.get("cve_modified"), start, end, cve.get("nvd_severity"), cve.get("nvd_score"), cve.get("nvd_severity_v3"), cve.get("nvd_score_v3")])
      except Exception as e:
        print(f"error {e} with cve {cve['cve']}, skipping...")


In [18]:
output = "drive/My Drive/cves_temp/aggregated-severity-cat-google-play.csv"

to_csv(cves, output)

In [19]:
from datetime import datetime

output = "drive/My Drive/cves_temp/bulletin_dates.csv"

bulletin_dates = [(path.basename(url), date.strftime("%Y-%m-%d")) for url, date in urls]

with open(output, "w") as csvfile:
  w = csv.writer(csvfile)
  w.writerow(["security_patch", "date"])
  for row in bulletin_dates:
    w.writerow(row)
